In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,235 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
# Define the URL of the CSV file in the S3 bucket
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

# Use SparkFiles to add the URL to Spark's file system
spark.sparkContext.addFile(url)

# Read the CSV data into a Spark DataFrame
home_sales = spark.read.csv("file://" + SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)

# Create a temporary view of the DataFrame
home_sales.createOrReplaceTempView("home_sales")
home_sales.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
result = spark.sql("""
    SELECT date_built AS year, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
    ORDER BY date_built
""")
result.show()




+----+---------+
|year|avg_price|
+----+---------+
|2010|296800.75|
|2011| 302141.9|
|2012|298233.42|
|2013|299999.39|
|2014|299073.89|
|2015|307908.86|
|2016|296050.24|
|2017|296576.69|
+----+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
""")
result.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("""
    SELECT date_built AS year_built, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE bedrooms = 3
    AND bathrooms = 3
    AND floors = 2
    AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
""")
result.show()



+----------+---------+
|year_built|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
import time

start_time = time.time()

result = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS avg_view
    FROM home_sales
    WHERE price >= 350000
""")
result.show()

end_time = time.time()
print("--- %s seconds ---" % (end_time - start_time))


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.9470546245574951 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')



True

In [ ]:

# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.


# Start the timer
start_time = time.time()

# Run the query on the cached data
result_cached = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS avg_view
    FROM home_sales
    WHERE price >= 350000
""")
result_cached.show()

# End the timer
end_time = time.time()

# Calculate and print the runtime
runtime_cached = end_time - start_time
print("--- Cached Runtime: %s seconds ---" % runtime_cached)


+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- Cached Runtime: 0.308046817779541 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
# Assuming you have a DataFrame named formatted_home_sales

#possible place/path: C:\Users\gfran\Desktop\ASU-VIRT-DATA-PT-03-2023-U-LOLC-1\Homework\Module-22\Starter_Code
# Define the path where you want to save the Parquet data
parquet_path = r'C:\Users\gfran\Desktop\ASU-VIRT-DATA-PT-03-2023-U-LOLC-1\Homework\Module-22\Starter_Code'  # Replace with your desired path

# Write the DataFrame as a partitioned Parquet file
df.write.mode("home_sales").partitionBy("date_built").parquet(parquet_path)



NameError: ignored

In [ ]:
# 11. Read the parquet formatted data.
# Define the path where the Parquet data is stored
parquet_path = r'C:\Users\gfran\Desktop\ASU-VIRT-DATA-PT-03-2023-U-LOLC-1\Homework\Module-22\Starter_Code'  # Replace with the actual path

# Read the Parquet data into a DataFrame
parquet_data = spark.read.parquet(parquet_path)


IllegalArgumentException: ignored

In [ ]:
# 12. Create a temporary table for the parquet data.
# Create a temporary view for the Parquet data
parquet_data.createOrReplaceTempView("parquet_data")


NameError: ignored

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

# Start the timer
start_time = time.time()

# Run the query on the Parquet DataFrame
result_parquet = spark.sql("""
    SELECT ROUND(AVG(view), 2) AS avg_view
    FROM parquet_data
    WHERE price >= 350000
""")
result_parquet.show()

# End the timer
end_time = time.time()

# Calculate and print the runtime
runtime_parquet = end_time - start_time
print("--- Parquet Runtime: %s seconds ---" % runtime_parquet)




AnalysisException: ignored

In [ ]:
# 14. Uncache the home_sales temporary table.
# Uncache the "home_sales" temporary table
spark.table("home_sales").unpersist()


In [ ]:
# 15. Check if the home_sales is no longer cached
# Check if the "home_sales" temporary table is cached
is_cached = spark.table("home_sales").is_cached
if not is_cached:
    print("The 'home_sales' table is no longer cached.")
else:
    print("The 'home_sales' table is still cached.")



The 'home_sales' table is no longer cached.


In [ ]:
# Define the path where you want to save the DataFrame
#save_path = r'C:\Users\gfran\Desktop\Homework2\Starter_Code'  # Replace with your desired path

# Save the DataFrame to the specified location
#your_dataframe.write.mode("overwrite").save(save_path)


NameError: ignored